### EDA: NYC Open Data Common Column Names 
Hack day Q2 2020

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import spacy

In [ ]:
common_columns_df = pd.read_csv("nyc_columns_common.csv", index_col=0)
common_columns_df.head()

In [ ]:
common_columns_df.describe()

### Investigate the distribution of field types
We should (probably?) only match field names if for columns with the same field type

In [ ]:
common_columns_df.field_type.value_counts()

In [ ]:
# Based on the above, I would recommend the following mapping to identify similar field types
def clean_field_types(column):
    new_column = column.str.lower().str.replace('_', ' ')
    return new_column.replace('calendar date', 'date')

common_columns_df.field_type = clean_field_types(common_columns_df.field_type)

In [ ]:
# Now these field types *appear* unique, so we can match on these first
common_columns_df.field_type.value_counts()

### Let's investigate columns with field_type=date

In [ ]:
# Most common field names
date_columns_df = common_columns_df[common_columns_df['field_type'] == 'date'].copy()
date_columns_df.field_name.value_counts()

The main thing I notice here is that we need a simple text matching for things like `closed_date` <--> `date_closed`. The problem I foresee running into is that with some of the less-common columns like `event_date` we can try to map it onto something more common, but we don't know what other columns are in the dataset(s) along with it so we can't guarantee an injective mapping onto the column names and could end up with duplicates. 

In [ ]:
# Most common column names w/ type='date'
fig, ax = plt.subplots(1, 1, figsize=(20, 6))
sns.barplot(x=date_columns_df.field_name.value_counts().index, y=date_columns_df.field_name.value_counts().values)
plt.xticks(rotation=90);

In [ ]:
# Rank field names by number of appearances
field_name_counts = common_columns_df.field_name.value_counts()
field_name_counts

# Visualize top field names by # occurences
n = 25

fig, ax = plt.subplots(1, 1, figsize=(8, 4.5))
sns.barplot(x=field_name_counts.iloc[:n].values, y=field_name_counts.iloc[:n].index, ax=ax)
plt.tight_layout()

### Explore most common human-readable (Socrata) names
Within just these, it seems that zip & postal code should be investigated as a possible a match (in this specific case you'd need to watch out for 3 vs. 5-digit zips). 

In [ ]:
name_counts = common_columns_df.name.value_counts()
name_counts

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4.5))
sns.barplot(x=name_counts.iloc[:25].values, y=name_counts.iloc[:25].index, ax=ax)
plt.tight_layout()

In [ ]:
# from spacy.matcher import Matcher

# nlp = spacy.load("en_core_web_sm")
# matcher = spacy.matcher.Matcher(nlp.vocab)

# pattern = [{"LOWER": "date"}, {"IS_PUNCT": True}, {"LOWER": "start"}]
# matcher.add("date", None, pattern)

# doc = nlp("date")
# matches = matcher(doc)

# save = []
# for match_id, start, end in matches:
#     string_id = nlp.vocab.strings[match_id]  # Get string representation
#     span = doc[start:end]  # The matched span
#     print(match_id, string_id, start, end, span.text)

# print(matches)

# color_patterns = [nlp(text) for text in ('red', 'green', 'yellow')]
# product_patterns = [nlp(text) for text in common_date_cols.index]
# material_patterns = [nlp(text) for text in ('silk', 'yellow fabric')]

# matcher = spacy.matcher.PhraseMatcher(nlp.vocab)
# matcher.add('COLOR', None, *color_patterns)
# matcher.add('PRODUCT', None, *product_patterns)
# matcher.add('MATERIAL', None, *material_patterns)

# doc = nlp(' '.join(common_date_cols.index))
# matches = matcher(doc)
# for match_id, start, end in matches:
#     rule_id = nlp.vocab.strings[match_id]  # get the unicode ID, i.e. 'COLOR'
#     span = doc[start : end]  # get the matched slice of the doc
#     print(rule_id, span.text)